In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


<center><h1>*Speed Up Vaccine Rollout</h1></center>
<center><h2>*Update on 12-06-2021*</h2></center>

In [ ]:
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# offline plotly
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.graph_objs as go
# interactive plots
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# color pallette
cnf = '#393e46' # confirmed - grey
dth = '#ff2e63' # death - red
rec = '#21bf73' # recovered - cyan
act = '#fe9801' # active case - yellow
from datetime import datetime as dt
from math import pi
#output_notebook()
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
import gc
gc.collect()


In [ ]:
#Covid from worldmeter
df_vaccine = pd.read_csv("/kaggle/input/covid-world-vaccination-progress/country_vaccinations.csv")
#use_col = ['Country','Cases','NCases','Deaths','NDeaths','Recovered','NewRecovered']
df = pd.read_csv("/kaggle/input/worldmeter/20210328.csv",index_col=0)
remove_row = ['World','North America','Europe','Asia','South America','Africa']
df = df[~df['Country'].isin(remove_row)]
df = df.sort_values(by = 'Cases',ascending=False).reset_index(drop=True)
df = df.drop(columns='#')
df.fillna(0,inplace=True)
remove_row = ['World','North America','Europe','Asia','South America','Africa']

country_vaccine = df_vaccine.groupby(["country", "iso_code", "vaccines"])['total_vaccinations', 
                                                                       'total_vaccinations_per_hundred',
                                                                      'daily_vaccinations',
                                                                      'daily_vaccinations_per_million',
                                                                      'people_vaccinated',
                                                                      'people_vaccinated_per_hundred'
                                                                      ].max().reset_index()
country_vaccine.columns = ["Country", "iso_code", "Vaccines", "Total vaccinations", "Percent", "Daily vaccinations", 
                           "Daily vaccinations per million", "People vaccinated", "People vaccinated per hundred"]
df['Country'] = df['Country'].replace({'USA': 'United States'})
df['Country'] = df['Country'].replace({'UAE': 'United Arab Emirates'})
country_vaccine['Country'] = country_vaccine.replace({'United Arab Emirates':'United Arab Emirates'})
Combined_vaccine_world = pd.merge(df,country_vaccine,on='Country')

In [ ]:
df_temp = country_vaccine.copy()

df_temp['Is_UAE'] = df_temp['Country']=='UAE'
df_temp['Is_India'] = df_temp['Country']=='India'

df_temp['TotalvaccinationsPer1000'] = df_temp['Total vaccinations'] / 1000
df_temp['DailyPervaccinations_1000'] = df_temp['Daily vaccinations'] / 1000


def plot_topn(col, n , xa , xy, title):
    df_f = df_temp.sort_values(col, ascending=False).head(n)
    fig = px.bar(df_f, x=col, y='Country', text=col, 
                 orientation='h',hover_data=["Total vaccinations","Daily vaccinations","People vaccinated"],color='Is_India'
                ,color_discrete_sequence=['#FFA500', '#393e46'],opacity=0.8,)
    fig.update_layout(title=title, xaxis_title=xa, yaxis_title=xy, 
                      yaxis_categoryorder = 'total ascending',
                      showlegend=False,template = 'plotly_white')
    fig.show()
    

def plot_lown(col, n , xa , xy, title):
    df_f = df_temp.sort_values(col, ascending=False).head(n)
    fig = px.bar(df_f, x=col, y='Country', text=col, 
                 orientation='h',hover_data=["Total vaccinations","Daily vaccinations","Daily vaccinations per million"],color='Is_India'
                ,color_discrete_sequence=['#FFA500', '#393e46'],opacity=0.8,)
    fig.update_layout(title=title, xaxis_title=xa, yaxis_title=xy, 
                      yaxis_categoryorder = 'total ascending',
                      showlegend=False,template = 'plotly_white')
    fig.show()
    


    
def plot_count100(col, n , xa , xy, title):
    df_f = df_temp.sort_values(col, ascending=False).tail(n)
    fig = px.bar(df_f, x=col, y='Country', text=col, 
                 orientation='h',hover_data=["Total vaccinations","Daily vaccinations","People vaccinated per hundred"],color='Is_India'
                ,color_discrete_sequence=['#FFA500', '#393e46'],opacity=0.8,)
    fig.update_layout(title=title, xaxis_title=xa, yaxis_title=xy, 
                      yaxis_categoryorder = 'total ascending',
                      showlegend=False,template = 'plotly_white')
    fig.show()
    
    
#4.1 Popultion Per Million
#Combined_vaccine_world['Population'] = Combined_vaccine_world['Population'].astype(str).replace('\.0', '', regex=True)
Combined_vaccine_world['Population_Million'] = Combined_vaccine_world['Population'] / 1000000
#Combined_vaccine_world['Population'] = Combined_vaccine_world['Population'].astype(str).replace('\.0', '', regex=True)



fig41 = px.scatter(Combined_vaccine_world, x='Population', y='Daily vaccinations per million', color='Population',size='Population',
                 hover_name="Country",hover_data=['Country','CPM','TPM','Population'],
           log_x=True, log_y=True, title='Total Population Vs Daily vaccinations per million (Per Million), Size Population',
           color_continuous_scale=px.colors.sequential.Plasma,size_max = 30,height =600,template = 'simple_white'
      ,opacity=1
           )    

#4.2 Popultion vs vaccination

fig42 = px.scatter(Combined_vaccine_world, x='Population', y='Total vaccinations', color='Population',size='Population',
                 hover_name="Country",hover_data=['Country','CPM','TPM','Population'],
           log_x=True, log_y=True, title='Country Total Population Vs Country Total People Vaccinated , Size Population',
           color_continuous_scale=px.colors.sequential.Plasma,size_max = 30,height =600,template = 'simple_white'
      ,opacity=1
           )    

#Combined_vaccine_world['Population'] = Combined_vaccine_world['Population'].astype(str).replace('\.0', '', regex=True)


#4.3 Cases vs vaccination

fig43 = px.scatter(Combined_vaccine_world, x='CPM', y='Total vaccinations', color='Population',size='Population',
                 hover_name="Country",hover_data=['Country','CPM','TPM','Population'],
           log_x=True, log_y=True, title='Cases Per Million Vs Total Vaccinated , Size Population',
           color_continuous_scale=px.colors.sequential.Plasma,size_max = 30,height =600,template = 'simple_white'
      ,opacity=1
           )    

In [ ]:
plot_topn('Total vaccinations', 15,'Total vaccinations count','Country',"Total People Vaccinated Per Country")

In [ ]:
plot_lown('Daily vaccinations', 15,'Daily vaccinations count','Country',"Daily Vaccinations Count")

In [ ]:
fig41.update_coloraxes(colorscale=px.colors.sequential.Cividis_r)
fig41.update(layout_coloraxis_showscale=True)
fig41.show()
fig42.update_coloraxes(colorscale=px.colors.sequential.Cividis_r)
fig42.update(layout_coloraxis_showscale=True)
fig42.show()
fig43.update_coloraxes(colorscale=px.colors.sequential.Cividis_r)
fig43.update(layout_coloraxis_showscale=True)
fig43.show()

In [ ]:
bar_chart = Combined_vaccine_world.copy()
bar_chart[['Vaccine_1','Vaccine_2','Vaccine_3','Vaccine_4','Vaccine_5','Vaccine_6']] = bar_chart['Vaccines'].str.split(',',expand=True)
bar_chart = bar_chart.drop(columns='Vaccines')
bar_chart['Population'] = bar_chart['Population'].apply(int) 
bar_chart['Total vaccinations'] = bar_chart['Total vaccinations'].replace(np.nan, 0)
bar_chart['People vaccinated'] = bar_chart['People vaccinated'].replace(np.nan, 0)
bar_chart['Total vaccinations'] = bar_chart['Total vaccinations'].apply(int) 
bar_chart['People vaccinated'] = bar_chart['People vaccinated'].apply(int) 
bar_chart = bar_chart[['Country','Cases','Vaccine_1','Vaccine_2','Vaccine_3','Vaccine_4','Vaccine_5','Vaccine_6','Total vaccinations','Population']]
bar_chart.sort_values(by='Cases',ascending=False).style.background_gradient(cmap='plasma_r')

In [ ]:

fig1 = px.treemap(Combined_vaccine_world, path = ['Vaccines', 'Country'], values = 'Total vaccinations',color_continuous_scale='RdBu',
                 color='Total vaccinations',
                title="Total Vaccinations Per Country, Grouped By Vaccine Name , color differance by total vaccinaion count")
fig1.show()


In [ ]:
fig2 = px.treemap(Combined_vaccine_world, path = ['Continent','Vaccines', 'Country'], values = 'Total vaccinations',color_continuous_scale='RdBu',
                 color_continuous_midpoint=np.average(Combined_vaccine_world['Total vaccinations']),color='Total vaccinations',
                title="Total Vaccinations Per Country (Continent), Grouped By Vaccine Name , color differance by total vaccinaion count")
fig2.show()


In [ ]:
fig3 = px.treemap(Combined_vaccine_world, path = ['Country', 'Vaccines'], values = 'Total vaccinations',color_continuous_scale='RdBu',
                 color_continuous_midpoint=np.average(Combined_vaccine_world['Population_Million']),color='Population_Million',
                title="Vaccination Group , color diff coutry population avg")

fig3.show()



In [ ]:
fig4 = px.treemap(Combined_vaccine_world, path = ['Continent','Country', 'Vaccines'], values = 'Total vaccinations',color_continuous_scale='RdBu',
                 color_continuous_midpoint=np.average(Combined_vaccine_world['Population_Million']),color='Population_Million',
                title="Vaccination Group (Continent) , color diff coutry population avg")

fig4.show()

In [ ]:
## Will update Soon ..... !!!